In [1]:
import os
import sys
import  urllib.request
import pandas as pd
from tqdm import tqdm
import numpy as np
from pymol import *

In [2]:
aa_dict = {'GLY':'G','ALA' :'A','VAL': 'V','LEU': 'L','ILE': 'I','PRO': 'P','PHE' :'F','TYR' :'Y','TRP' :'W',
'SER' :'S','THR': 'T','CYS': 'C','MET': 'M','ASN': 'N','GLN' :'Q','ASP': 'D', 'GLU' :'E','LYS' :'K','ARG' :'R', 'HIS' :'H',
'NAG':'X', 'DG':'X','DT':'X','DC':'X','DA':'X','UNK':'X',
'GLX':'X'}

In [3]:

def get_chain_ca_coordinates(pdb_path, chain_id):
    """
    获取指定链上的所有氨基酸残基的 Cα（alpha-carbon）坐标
    """

    cmd.delete('all')
    # cmd.load('./data/PDBs/{}.pdb'.format(pdb_name))
    cmd.load(pdb_path)
    cmd.remove('solvent')
    cmd.remove('hetatm')

    pos_dict = dict()  # 临时存储数据
    model = cmd.get_model(f"chain {chain_id} and name CA")  # 获取 Cα 原子数据
    for atom in model.atom:
        resn = atom.resn   # 残基名称
        resi = atom.resi   # 残基编号
        x, y, z = atom.coord  # Cα 原子的坐标
        # stored_list.append([resn, resi, x, y, z])

        ID = '{}-{}'.format(resi, aa_dict[resn])
        pos = [x, y, z]
        pos_dict[ID] = pos

    return pos_dict



In [4]:
aa_coor_dict = dict()


##Training
data = pd.read_csv('./S4169.csv').values.tolist()
# data = pd.read_csv('./S4169.csv').values.tolist()
for item in tqdm(data):
    pdb_name = item[0]
    biounit_chains = list(item[1].replace('_',''))
    mutation = item[4]
    ddg = item[5]

    mut_chain = mutation[0]
    src_aa = mutation[2]
    tgt_aa = mutation[-1]
    pos = mutation[3:-1]

    for chain in biounit_chains:
        subunit_ID = pdb_name + '_' + chain
        if subunit_ID not in aa_coor_dict.keys():
            pos_data = get_chain_ca_coordinates('./PDBs/{}.pdb'.format(pdb_name),chain) # 获取alpha碳原子坐标
            aa_coor_dict[subunit_ID] = pos_data
    
    ##Check if mutation information is wrong
    src_aa_pos = '{}-{}'.format(pos,src_aa)
    mut_subunit = pdb_name + '_' + mut_chain
    if src_aa_pos not in aa_coor_dict[mut_subunit].keys():
        print("Something is wrong for {}".format(item))

np.save('./S4169_aa-position.npy',aa_coor_dict)
print('S4169 done!')


  0%|          | 0/4169 [00:00<?, ?it/s]

100%|██████████| 4169/4169 [00:35<00:00, 116.29it/s]


S4169 done!


In [5]:
aa_coor_dict = dict()


##Test 2
no_error_count = 0
# data = pd.read_csv('./data/SKEMPI2/AB-Bind-Database/AB_645.csv').values.tolist() 
data = pd.read_csv('./AB-Bind-Database/AB_645.csv').values.tolist() 
for item in tqdm(data):
    pdb_name = item[0]
    # if 'HM_' in pdb_name:
    #     pdb_name = item[0].replace('HM_','')
    biounit_chains = list(item[1].replace('_',''))
    mutation = item[4]
    ddg = item[5]

    mut_chain = mutation[0]
    src_aa = mutation[2]
    tgt_aa = mutation[-1]
    pos = mutation[3:-1]

    for chain in biounit_chains:
        subunit_ID = pdb_name + '_' + chain

        if subunit_ID not in aa_coor_dict.keys():
            pos_data = get_chain_ca_coordinates('./AB-Bind-Database/{}.pdb'.format(pdb_name),chain) # 获取alpha碳原子坐标
            aa_coor_dict[subunit_ID] = pos_data

    ##Check if mutation information is wrong
    src_aa_pos = '{}-{}'.format(pos,src_aa)
    mut_subunit = pdb_name + '_' + mut_chain

    if src_aa_pos.upper() not in aa_coor_dict[mut_subunit].keys():
        print("Something is wrong for {}-{}-{}".format(item[0], item[1],item[4]))
    else:
        no_error_count += 1


print(no_error_count)
np.save('./S645_aa-position.npy',aa_coor_dict)
print('S645 done!')


  0%|          | 0/645 [00:00<?, ?it/s]

100%|██████████| 645/645 [00:03<00:00, 176.80it/s]

645
S645 done!
